In [1]:
# Check GPU
!nvidia-smi

# Fresh install Ultralytics (YOLO) + torch (Colab usually bundles torch already)
!pip -q install --upgrade ultralytics==8.* rich

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Sun Nov 23 17:11:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             14W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
import zipfile
# Load Data
data_dir = 'yolo_dataset'

# Check if the dataset folder exists, if not, unzip the dataset
if not os.path.exists(data_dir):
    zip_file_path = 'yolo_dataset.zip'
    extract_dir = 'yolo_dataset'

    if not os.path.exists(extract_dir):
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file_path} to {extract_dir}")
    else:
        print(f"Directory {extract_dir} already exists, skipping extraction.")

Extracted yolo_dataset.zip to yolo_dataset


In [3]:
%%writefile train_potholes.py

import argparse
import shutil
import random
import time
from pathlib import Path
from collections import defaultdict
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.metrics import precision_recall_fscore_support, average_precision_score
import yaml

# -----------------------------
# Config defaults (can be overridden by CLI)
# -----------------------------
BASE = Path.cwd()
DATA_ROOT = Path("/content/drive/MyDrive/Faizan/dataset")  # Your dataset path

IMG_DIR = DATA_ROOT / "IMG"
LBL_PSEUDO_DIR = DATA_ROOT / "TXT_pseudo"   # Labels directory
WORK_DS = BASE / "yolo_dataset"             # Output YOLO dataset (images/labels split into train/val)
DATA_YAML = WORK_DS / "data.yaml"           # Path to save data.yaml
RNG_SEED = 42

CLASS_MAP = {0: "dry", 1: "water-filled"}
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

# Helpers
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def list_images(root: Path):
    return [p for p in root.rglob("*") if p.is_file() and p.suffix.lower() in IMAGE_EXTS]

def read_label_lines(p: Path):
    if not p.exists():
        return []
    try:
        return [ln.strip() for ln in p.read_text(encoding="utf-8").splitlines() if ln.strip()]
    except UnicodeDecodeError:
        return [ln.strip() for ln in p.read_text(encoding="latin-1", errors="ignore").splitlines() if ln.strip()]

def parse_yolo_line(line: str):
    parts = line.split()
    if len(parts) != 5:
        return None
    try:
        cls = int(float(parts[0])); x, y, w, h = map(float, parts[1:])
        return cls, x, y, w, h
    except Exception:
        return None

def image_has_classes(txt_path: Path):
    s = set()
    for ln in read_label_lines(txt_path):
        parsed = parse_yolo_line(ln)
        if parsed is None:
            continue
        cls, *_ = parsed
        s.add(int(cls))
    return s

def stratified_split(stems, stem_to_classes, val_ratio=0.2, seed=RNG_SEED):
    rng = random.Random(seed)
    buckets = defaultdict(list)
    for st in stems:
        sig = tuple(sorted(stem_to_classes.get(st, set())))
        buckets[sig].append(st)

    train, val = [], []
    for sig, items in buckets.items():
        rng.shuffle(items)
        k = int(round(len(items) * val_ratio))
        val.extend(items[:k])
        train.extend(items[k:])
    rng.shuffle(train); rng.shuffle(val)
    return train, val, buckets

def copy_split_files(stems, img_by_stem, lbl_dir, split_name, work_root):
    img_out_dir = work_root / "images" / split_name
    lbl_out_dir = work_root / "labels" / split_name
    ensure_dir(img_out_dir); ensure_dir(lbl_out_dir)

    copied = 0
    missing_labels = []
    for st in stems:
        img_path = img_by_stem.get(st)
        if img_path is None:
            continue
        dst_img = img_out_dir / img_path.name
        if not dst_img.exists():
            shutil.copy2(img_path, dst_img)

        src_lbl = lbl_dir / f"{st}.txt"
        if src_lbl.exists():
            dst_lbl = lbl_out_dir / src_lbl.name
            if not dst_lbl.exists():
                shutil.copy2(src_lbl, dst_lbl)
        else:
            dst_lbl = lbl_out_dir / f"{st}.txt"
            if not dst_lbl.exists():
                dst_lbl.write_text("")
            missing_labels.append(str(src_lbl))
        copied += 1
    return copied, missing_labels

# Write data.yaml
def write_data_yaml(path: Path, work_root: Path):
    d = {
        "path": str(work_root),  # Root directory
        "train": str(work_root / "images" / "train"),  # Path to training images
        "val": str(work_root / "images" / "val"),  # Path to validation images
        "names": {0: CLASS_MAP[0], 1: CLASS_MAP[1]},  # Class names mapping
    }
    # Saving the YAML file
    with open(path, 'w') as file:
        yaml.dump(d, file, default_flow_style=False)

# Main function
def main():
    parser = argparse.ArgumentParser(description="Train Ultralytics YOLO on potholes (dry vs water-filled).")
    parser.add_argument("--model", default="yolov8m.pt", type=str)
    parser.add_argument("--epochs", default=100, type=int)
    parser.add_argument("--imgsz", default=640, type=int)
    parser.add_argument("--batch", default=16, type=int)
    parser.add_argument("--val_ratio", default=0.2, type=float)
    parser.add_argument("--project", default=str(BASE / "runs_yolo"), type=str)
    parser.add_argument("--name", default="potholes_yolov8m", type=str)
    parser.add_argument("--patience", default=50, type=int)
    parser.add_argument("--seed", default=RNG_SEED, type=int)
    parser.add_argument("--workers", default=8, type=int)
    parser.add_argument("--freeze", default=0, type=int)
    parser.add_argument("--cos_lr", default=True, action="store_true")
    parser.add_argument("--no_cos_lr", dest="cos_lr", action="store_false")
    parser.set_defaults(cos_lr=True)
    args = parser.parse_args()

    random.seed(args.seed); np.random.seed(args.seed)

    # Pre-flight checks
    assert IMG_DIR.exists(), f"Images dir not found: {IMG_DIR}"
    assert LBL_PSEUDO_DIR.exists(), f"Pseudo labels dir not found: {LBL_PSEUDO_DIR}"

    # Gather images and build stem maps
    img_files = list_images(IMG_DIR)
    img_by_stem = {p.stem: p for p in img_files}
    stems = sorted(img_by_stem.keys())

    # Compute image-level class presence to stratify
    stem_to_classes = {}
    for st in stems:
        txt = LBL_PSEUDO_DIR / f"{st}.txt"
        stem_to_classes[st] = image_has_classes(txt)

    # Build WORK_DS structure and split
    if WORK_DS.exists():
        shutil.rmtree(WORK_DS)
    ensure_dir(WORK_DS)

    # Create train and val splits
    train_stems, val_stems, buckets = stratified_split(stems, stem_to_classes, val_ratio=args.val_ratio, seed=args.seed)

    _, miss_train = copy_split_files(train_stems, img_by_stem, LBL_PSEUDO_DIR, "train", WORK_DS)
    _, miss_val   = copy_split_files(val_stems,   img_by_stem, LBL_PSEUDO_DIR, "val",   WORK_DS)

    # Save split report
    split_report = {
        "total_images": len(stems),
        "train_images": len(train_stems),
        "val_images": len(val_stems),
        "buckets": {str(sig): len(items) for sig, items in buckets.items()},
        "missing_train_labels": len(miss_train),
        "missing_val_labels": len(miss_val),
    }
    (WORK_DS / "split_report.json").write_text(pd.Series(split_report).to_json(indent=2))
    print("\n=== SPLIT REPORT ===")
    for k,v in split_report.items():
        print(f"{k}: {v}")

    # Write data.yaml
    write_data_yaml(DATA_YAML, WORK_DS)
    print(f"\nWrote dataset YAML -> {DATA_YAML}")

    # Train YOLO
    print("\n=== TRAINING ===")
    model = YOLO(args.model)

    results = model.train(
        data=str(DATA_YAML),
        epochs=args.epochs,
        imgsz=args.imgsz,
        batch=args.batch,
        project=args.project,
        name=args.name,
        workers=args.workers,
        seed=args.seed,
        patience=args.patience,
        cos_lr=args.cos_lr,
        freeze=args.freeze,
        verbose=True,
    )

    # Validate
    print("\n=== VALIDATION ===")
    eval_results = model.val(data=str(DATA_YAML), imgsz=args.imgsz)

    # Retrieve mAP, precision, recall from eval_results
    # Access the metrics from `results_dict`
    precision = eval_results.results_dict['metrics/precision(B)']
    recall = eval_results.results_dict['metrics/recall(B)']
    mAP50 = eval_results.results_dict['metrics/mAP50(B)']
    mAP50_95 = eval_results.results_dict['metrics/mAP50-95(B)']

    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, mAP50: {mAP50:.4f}, mAP50-95: {mAP50_95:.4f}")

    # Inference time
    start_time = time.time()
    sample_imgs = sorted([p for p in (WORK_DS / "images" / "val").glob("*") if p.suffix.lower() in IMAGE_EXTS])[:8]
    _ = model.predict(source=[str(p) for p in sample_imgs], imgsz=args.imgsz)
    inference_time = time.time() - start_time
    print(f"Inference Time for 8 images: {inference_time:.2f} seconds")

if __name__ == "__main__":
    main()


Writing train_potholes.py


In [4]:
!python train_potholes.py \
  --model yolov8n.pt \
  --epochs 500 \
  --imgsz 640 \
  --batch 16 \
  --val_ratio 0.2 \
  --project /content/runs_yolo \
  --name potholes_yolov8n

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

=== SPLIT REPORT ===
total_images: 713
train_images: 570
val_images: 143
buckets: {'(1,)': 250, '(0,)': 463}
missing_train_labels: 0
missing_val_labels: 0

Wrote dataset YAML -> /content/yolo_dataset/data.yaml

=== TRAINING ===
Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropou